In [288]:
%pylab
import scipy.interpolate
import desimodel.io
def get_radius_mm(theta):
    """
    Returns an array of radii in mm given an array of radii in degrees using the platescale data 
    relative to the center of the focal plane as (0,0). Supports scalar and vector inputs.
    Parameters
    ----------
    theta: An array that represents the angle from the center of the focal plane
    """
    if(np.isscalar(theta)):
        platescale = desimodel.io.load_platescale()
        # Uses a quadratic one-dimensional interpolation to approximate the radius in degrees versus radius in mm
        fn = scipy.interpolate.interp1d(platescale['theta'], platescale['radius'], kind = 'quadratic')
        radius = float(fn(theta))
        return radius
    else:
        platescale = desimodel.io.load_platescale()
        # Uses a quadratic one-dimensional interpolation to approximate the radius in degrees versus radius in mm
        fn = scipy.interpolate.interp1d(platescale['theta'], platescale['radius'], kind = 'quadratic')
        radius = []
        for x in theta:
            radius_mm = float(fn(x))
            radius.append(radius_mm) 
        radius_array = np.array(radius)
        return radius_array

def get_radius_deg(x, y):
    """
    Returns the radius in degrees given x, y coordinates using the platescale data
    Parameters
    ----------
    x: The x coordinate in mm of a location on the focal plane
    y: The y coordinate in mm of a location on the focal plane
    """
    import desimodel.io
    radius = np.sqrt(x**2 + y**2)
    platescale = desimodel.io.load_platescale()
    # Plots are used for debugging. 
    #plot(platescale['radius'], platescale['theta'], 'k.')
    #plot(platescale['radius'], platescale['radial_platescale'], 'k.')
    fn = scipy.interpolate.interp1d(platescale['radius'], platescale['theta'], kind = 'quadratic')
    degree = float(fn(radius))
    return degree

def cartesian_to_polar_angle(x, y):
    """
    Given cartesian coordinates, this function returns the polar angle in degrees
    for use in polar coordinates
    Parameters
    ----------
    x: The x coordinate in mm of a location on the focal plane
    y: The y coordinate in mm of a location on the focal plane
    """
    # Prevents division by 0
    if(x == 0):
        if(y > 0):
            return 90
        else:
            return 270
    # Case for quadrant 1
    elif(x > 0 and y >= 0):
        return arctan(y / x) * 180 / pi
    # Case for quadrants 2 and 3
    elif(x < 0):
        return (arctan(y / x) * 180 / pi) + 180
    # Case for quadrant 4
    elif(x > 0 and y <= 0):
        return (arctan(y / x) * 180 / pi) + 360
    else:
        raise ValueError

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [289]:
#degree = get_radius_deg(333.738, 217.766)
#print(degree)
#radius = get_radius_mm(1.5731300326614939)

testradii = [1.5731300326614939, 1.5]
radius = get_radius_mm(testradii)
print(radius)

[ 398.50104567  378.53678987]


In [290]:
def plot_focal_plane_deg(): 
    """
    Plots the focal plane in degrees using the gfa Table object read in with the astropy library
    """
    %pylab
    import desimodel.io
    # ONLY USED HERE RIGHT NOW BECAUSE LOAD_GFA() HAS NOT BEEN PUSHED
    gfa = desimodel.io.load_gfa()
    
    pyplot.polar()
    title('GFA Overhead View in Degrees')
    # Theta is in degrees
    theta = gfa['Q']
    degree = gfa['RADIUS_DEG']
    print(gfa)
   
    for i in range(40):
        thetaradians = theta[i] * pi / 180
        pyplot.polar(thetaradians, degree[i], 'k.')
        
        # Used for debugging to print the angle in radians and the radius in degrees
        #print(str(theta[i]) + ", " + str(degree[i]))
        
                 

In [291]:
plot_focal_plane_deg()

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
PETAL CORNER       X        ...       Q         RADIUS_DEG    RADIUS_MM  
                   mm       ...    degrees                               
----- ------ -------------- ... ------------- ------------- -------------
    0      0  116.279135121 ...  287.31935812 1.54426530071 390.595017971
    0      1  121.005023896 ... 287.345026987  1.5999816328 405.886737469
    0      2   91.784287159 ...  283.02060759 1.60541232205 407.384285137
    0      3  87.0587074141 ... 282.826719301 1.54995479057 392.150643917
    1      0  313.248421438 ...  323.31935812 1.54426530071 390.595017971
    1      1  325.620626724 ... 323.345026987  1.5999816328 405.886737469
    1      2  307.552931346 ...  319.02060759 1.60541232205 407.384285137
    1      3  295.180417053 ... 318.826719301 1.54995479057 392.150643917
    2      0  390.567457688 ...  359.31935812 1.54426530071 390.595017971
    2      1  40

In [292]:
# ADD TO FOCALPLANE.PY FILE

%pylab
def xy2radec(telra, teldec, x, y):
    """
    Returns the new RA and Dec of an x, y position on the focal plane
    in the sky given an arbitrary telescope pointing in RA and Dec
    Parameters
    ----------
    telra: a float signifying the telescope's RA pointing in degrees
    teldec: a float signifying the telescope's Dec pointing in degrees
    x: The x coordinate in mm of a location on the focal plane
    y: The y coordinate in mm of a location on the focal plane
    """
    from math import atan2, acos
    
    # radial distance on the focal plane in degrees
    r_deg = get_radius_deg(x, y)
    print(r_deg)
    # q signifies the angle the position makes with the +x-axis of focal plane
    q = cartesian_to_polar_angle(x, y)
    print(q)
    
    coord = numpy.zeros(shape=(3,1))
    coord[0] = 1
    
    # Clockwise rotation around the z-axis by the radial distance to a point on the focal plane in radians
    zrotate = numpy.zeros(shape=(3,3))
    r_rad = math.radians(r_deg)
    zrotate[0] = [cos(r_rad), sin(r_rad), 0]
    zrotate[1] = [-sin(r_rad), cos(r_rad), 0]
    zrotate[2] = [0, 0, 1]

    # Counter-clockwise rotation around the x-axis
    xrotate = numpy.zeros(shape=(3,3))
    q_rad = math.radians(q)
    xrotate[0] = [1, 0, 0]
    xrotate[1] = [0, cos(q_rad), -sin(q_rad)]
    xrotate[2] = [0, sin(q_rad), cos(q_rad)]
    
    # Counter-clockwise rotation around y axis by declination of the tile center
    decrotate = numpy.zeros(shape=(3,3))
    teldec_rad = math.radians(teldec)
    decrotate[0] = [cos(teldec_rad), 0, -sin(teldec_rad)]
    decrotate[1] = [0, 1, 0]
    decrotate[2] = [sin(teldec_rad), 0, cos(teldec_rad)]
    
    # Counter-clockwise rotation around the z-axis by the right ascension of the tile center
    rarotate = numpy.zeros(shape=(3,3))
    telra_rad = math.radians(telra)
    rarotate[0] = [cos(telra_rad), -sin(telra_rad), 0]
    rarotate[1] = [sin(telra_rad), cos(telra_rad), 0]
    rarotate[2] = [0, 0, 1]
    
    coord1 = matmul(zrotate, coord)
    coord2 = matmul(xrotate, coord1)
    coord3 = matmul(decrotate, coord2)
    coord4 = matmul(rarotate, coord3)
    
    ra_rad = atan2(coord4[1], coord4[0])
    dec_rad = (pi / 2) - acos(coord4[2] / sqrt((coord4[0]**2) + (coord4[1]**2) + (coord4[2]**2)))
    ra_deg = math.degrees(ra_rad)
    dec_deg = math.degrees(dec_rad)
    # Value can be 360, which should be 0
    ra = ra_deg % 360
    
    
    # MAKE THIS CHANGE AND COMMIT TO GITHUB IN NEW BRANCH BECAUSE RA SHOULD BE BETWEEN 0 AND 360
    if(ra == 360):
        ra = 0 
        
        
        
    return ra, dec_deg
    
    
newra, newdec = xy2radec(8.37, -10.65, -138.345, -333.179)
#simple test case of only moving in ra
#newra1, newdec1 = xy2radec(0, 0, 400, 0)
#newra2, newdec2 = xy2radec(0, 0, 0, 400)
print(newra)
print(newdec)
#print(newra1)
#print(newdec1)
#print(newra2)
#print(newdec2)


Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
1.4342182409681465
247.450443362
8.927313423598427
-9.324956250231294


In [293]:
from astropy.io import fits
from astropy.table import Table
#import numpy as np
import desimodel.io
#%pylab

tile = Table.read('tile_00612.fits', 'FIBER_ASSIGNMENTS')

#- File bug: TILERA, TILEDEC are arrays of identical values instead of scalars;
#- just grab element 0
telra = tile.meta['TILERA'][0]
teldec = tile.meta['TILEDEC'][0]
print(telra)
print(teldec)

plot(tile['XFOCAL_DESIGN'], tile['YFOCAL_DESIGN'], 'b.')

clf()
plot(tile['RA'], tile['DEC'], 'b.')
tile

8.37
-10.65


FIBER,LOCATION,NUMTARGET,PRIORITY,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,RA,DEC,XFOCAL_DESIGN,YFOCAL_DESIGN,BRICKNAME
,,,,,,,,deg,deg,mm,mm,
int32,int32,int16,int32,int64,int64,int64,int64,float64,float64,float32,float32,str8
0,95,11,3200,600548863582474091,65537,0,0,8.40634632111,-9.93649291992,-8.77733,-174.925,0084m100
1,62,6,3000,5844768686203625466,131074,0,0,8.56390190125,-9.99551010132,-46.782,-160.322,0084m100
2,102,7,0,3393924777442882107,4294967296,0,0,8.58276939392,-9.87040424347,-51.5125,-191.556,0084m097
3,82,16,3400,6166091188264142003,262148,0,0,8.48240280151,-9.95259666443,-27.1377,-170.94,0084m100
4,131,9,3400,5942221679052239683,262148,0,0,8.34638404846,-9.82890224457,5.72046,-201.849,0082m097
5,108,12,3000,8685456968741477546,131074,0,0,8.32260322571,-9.93105316162,11.4478,-176.283,0082m100
6,119,6,3200,6759846742251782601,65537,0,0,8.3402633667,-9.883644104,7.19154,-188.122,0082m100
7,64,7,3000,355041889027632097,131074,0,0,8.481508255,-10.0389022827,-26.8648,-149.509,0084m100


In [294]:
def radecdifference():
    """
    Reads in a file with data from 5000 fiberpositioners at a given telescope pointing
    and graphs a quiver plot and histogram to show the difference in calculations 
    """
    from astropy.io import fits
    from astropy.table import Table
    #import numpy as np
    import desimodel.io
    #%pylab

    tile = Table.read('tile_00612.fits', 'FIBER_ASSIGNMENTS')

    #- File bug: TILERA, TILEDEC are arrays of identical values instead of scalars;
    #- just grab element 0
    telra = tile.meta['TILERA'][0]
    teldec = tile.meta['TILEDEC'][0]
    ra = numpy.zeros(5000)
    dec = numpy.zeros(5000)
    # u and v are for the quiver plots, signifying the x and y components of the vectors respectively
    """u = numpy.zeros(5000)
    v = numpy.zeros(5000)"""
    
    for i in range(5000):
        ra[i], dec[i] = xy2radec(telra, teldec, tile['XFOCAL_DESIGN'][i], tile['YFOCAL_DESIGN'][i])
        """u[i] = ra[i] - tile['RA'][i]
        v[i] = dec[i] - tile['DEC'][i]"""
    u = ra - tile['RA']
    v = dec - tile['DEC']
    #q = quiver(tile['RA'], tile['DEC'], 10*u, 10*v)
    hist(sqrt((u*cos(tile['DEC'] * pi / 180))**2 + (v)**2)*3600)
radecdifference()



0.7145814065483651
267.127436675
0.6819035589068885
253.732773185
0.8073982078565614
254.948362442
0.7062962451693294
260.979221403
0.8216057315270823
271.623342216
0.7206362676647687
273.715538442
0.7670923182807993
272.189237049
0.6210480351920891
259.813376414
0.7088615094779935
255.787888313
0.8265840164088888
276.448870865
0.8218297739975851
273.768158219
0.7088705204790162
270.592475425
0.63410370775887
264.751295825
0.8433912552519518
267.458373001
0.6776327190831324
265.127263588
0.8666975543485691
267.652541619
0.8334667181458949
260.254251465
0.8688582934311277
274.185735494
0.7960425804215878
275.910228486
0.9007892486183848
271.791132643
0.6578483389275219
263.65384152
0.8056979776656967
267.592162747
0.7247393235797405
275.564231593
0.781001714555992
269.408856462
0.7744001617814836
263.098847669
0.7693763516281691
266.815447295
0.8112934990865779
262.552179818
0.8751965876769487
275.155526734
0.6942385171259263
269.236879129
0.7400813263370053
252.585130649
0.863510690271

1.3872360301852038
266.538080143
1.4272670277420472
274.726625005
1.3165025720854202
263.537694153
1.431312491263554
273.284130033
1.273291420326882
263.832669533
1.3349570673315987
265.316808168
1.3441372500520017
261.567389483
1.4786484326955016
270.401838015
1.4704690277872994
269.601262891
1.406765708691653
275.974575698
1.372903564570839
264.661116838
1.482870472055296
273.37463004
1.3712079696215453
264.022726377
1.2950386325958902
269.1484555
1.3927734658313424
267.932456431
1.3688857839007178
272.215219089
1.37163895647966
269.441094955
1.4432188160022734
271.98886662
1.508809606414025
275.237099683
1.4103255464322741
269.606091835
1.4457258382813845
277.039463662
1.4200167216415283
274.45709891
1.38108472863303
267.970514245
1.3331643780250033
271.91484786
1.3807980358340424
272.822532189
1.3067139973450088
262.63096605
1.3591593775127362
265.834092545
1.3619580156493014
266.9449067
1.3888341312000858
273.977879178
1.4321614325652032
266.258383896
1.3565937786506101
273.275291

1.4789635967988464
305.081136563
1.4110149486994423
299.564232261
1.2529922050239282
299.157924798
1.406327155504843
305.698110966
1.38338405571259
301.013755498
1.3686494330869416
299.044017733
1.3394925827277082
307.862109556
1.439078862054359
309.303112131
1.3694374699299758
306.515080138
1.4192753191323892
308.517373999
1.3568795230810717
297.878165914
0.9034577313500268
308.452746749
0.7357233393621962
311.170800039
0.6609910092375276
308.306461172
0.6389465942308578
291.366188427
0.6696262083603236
287.835410928
0.6218529705245072
299.971694026
0.8244585035664772
306.506812194
0.6989577947723653
306.425942754
0.834690612843114
302.111885992
0.7587898353825487
305.539698389
0.8531238302223231
303.671771424
0.8670655197836674
312.111713238
0.6581784152321103
296.422188149
0.6883832473820339
303.433250973
0.6997153956357596
305.397794379
0.6948036232208977
306.647476525
0.7971839682356802
301.570320292
0.8662957761631702
307.120029993
0.7834943869529639
300.952714097
0.7694672260446

1.5642202186785341
309.806288752
1.5227233801175473
302.724871564
1.2697637222731222
296.27672561
1.5592369859571447
307.277486344
1.2718939784010517
291.535010213
1.2552066998431723
292.918690425
1.5870182211137407
306.751773514
1.3722248702251856
296.034185593
1.3537520215181298
294.470038122
1.2291023883856462
293.312922114
1.3396310211101015
293.124190327
1.576041764746436
313.417276577
1.3298445144176578
291.885009596
1.21423326915574
288.309221151
1.2903019028096936
294.755963501
1.32376450212158
288.1575568
1.255054230208735
290.118927583
1.236086937914159
297.015431059
1.5694157991892244
309.075738549
1.6042161678774947
301.303198424
1.3248865944846853
295.560040691
1.353083691899244
291.174089357
1.2677509964606262
288.115639929
1.551620886700668
305.065109524
1.5371253744328643
310.03009356
1.5670448064694196
311.002719328
1.2899855394168427
290.061820371
1.5605612507571753
305.505404009
1.2369991347286597
291.473504359
1.3274443888910457
296.863111011
1.5007501188467978
305.

0.9562596378447283
350.491435817
0.9915983905299168
337.813061094
1.1447543278916221
356.526257569
0.943914522278313
335.517297911
1.01833734106906
350.440672997
0.9603211944002736
347.809012209
1.1074303270703107
352.658412369
1.1608207527349346
358.396936139
0.988578552267358
348.198681331
1.0079224049260964
347.336866845
1.0461400055206567
351.958260776
0.9752112685539653
340.434301575
0.9088103889193831
342.36298478
1.02315636391212
343.326903764
0.940405022629171
340.104263419
1.0852742852457768
349.82706417
0.9647386543113041
342.000873763
0.9977359927479151
351.363594625
1.208179876005891
358.392784985
0.9935600013603338
345.171162416
1.047638886377074
339.120581283
0.9965425152748603
342.310054229
0.8723569129610009
338.404517911
1.0176388347063001
336.703551505
1.0640911334262344
357.402886309
1.1008783552986978
352.113277896
1.016189073396759
346.284087953
1.1001596098933482
355.18400288
1.0527586341460518
353.473025125
1.1086623544522956
348.554037291
1.1001289144819761
358.

...

1.590076080743845
125.673173946
1.2264468731401754
117.09525548
1.581783960714262
128.935074218
1.2401553786615445
112.794927398
1.3600838789018872
115.026992237
1.534970562412021
123.125974423
1.3531135146836444
111.894257672
1.562515401076369
127.752054718
1.2916458849794417
115.965644644
1.5706462263510101
124.727780142
1.5181582465271541
121.93129579
1.2751261208210423
117.048653099
1.5588645096770932
122.038154099
1.2470005308883632
108.937216064
1.2738119794157532
110.739170231
1.6146310636602377
121.755397928
1.2121587195976005
114.452067894
1.384471434866474
116.09259054
1.3075313124158756
111.674332411
1.5552591004107184
132.907406371
1.5950500981320057
127.190660979
1.5944185069496277
129.769870183
1.4854542906628465
122.219925924
1.5252768540429673
123.923942009
1.3317227894567218
114.639829794
1.2679704733612953
109.270604427
1.1437442287372142
120.112935508
1.2080325584361695
132.512300418
1.2225932492450238
120.255467896
1.2881069186021659
131.908515111
1.246550580089459


0.6332034945114493
119.443925309
0.6653718367921008
121.347000427
0.7853735287813555
124.411272591
0.8825990028600463
124.82827613
0.8232791961114361
111.203035698
0.8126041114739465
125.688890211
0.7635842290655452
123.448588163
0.8124739969557035
115.776755763
0.666018401803908
108.961593009
1.1263999551227457
157.086115523
1.2775034426519587
157.322672295
1.2453000035109936
160.507097731
1.1332360149515737
165.315779718
1.0737838096341716
159.798673926
1.212785406853897
165.2401866
1.201778328106983
156.532224748
1.2233860480141083
164.496414858
1.1605275285183305
155.656702072
1.1965320595028937
160.794792129
1.165883751982142
170.897622044
1.2286182744314424
165.688725123
1.2072370525423595
172.545288214
1.2026908087733275
158.4887017
1.2254282690389384
162.558879728
1.2266036620541532
154.656396962
1.0942763028716256
158.567884083
1.2713077626595637
167.336860015
1.1578945831018943
158.459051848
1.2749050235504813
163.904667759
1.2963755535640789
170.280486278
1.1077015567560453


1.3540964225015188
193.772133895
1.3203316405193162
193.060672363
1.3880435910664684
194.086980857
1.325256900676422
191.317521002
1.3460728320178739
200.798545862
1.4476912833483044
194.332451879
1.3324492767341132
195.078093385
1.43423055853296
199.722352228
1.4596720460297348
202.608330841
1.4854408370873966
198.842915854
1.3936747927537074
197.068864717
1.3340297851833531
197.103806257
1.2590960740520454
191.466211659
1.3777771004750219
203.173477769
1.4721515733167874
203.714415091
1.3487304483841185
202.986120557
1.3334070037750165
199.79896646
1.3020936536817032
196.847867116
1.4771697120635006
197.508127843
1.4356914179994362
204.656696036
1.507821949530409
202.257930184
1.3522125711667325
199.590922313
1.4222190300567472
197.623976389
1.338969584287565
190.364736933
1.4594008373004097
200.070956546
1.3903827353141458
190.201916323
1.3973960383411024
190.849403742
1.3849282155001497
202.25615092
1.356498937163378
197.991457642
1.3997274571118974
198.606968677
1.4898975162035142

1.4155193200629856
206.833444881
1.1549151200139387
183.268408375
1.4273066820416678
210.980573528
1.07499945876033
191.148127188
1.1088843316050936
202.315582103
1.0752050345465023
201.393219698
0.8938641222859068
192.520520513
0.975197482880755
202.566077584
1.0264609217803573
203.204790771
0.9538859430305766
202.729906501
1.0570168585544344
206.746383875
0.9315465540178022
198.430025796
1.0750044934400418
208.469944546
0.9480893502218685
195.976010544
1.0242853723552587
192.687407647
1.1635058860392755
210.226435266
0.995267780096342
206.206303741
1.1234758059469112
208.107507388
1.0164940085790592
195.432597279
0.9844568785052931
195.758831019
1.0520984448918735
213.345345528
1.0017127829031092
201.660461094
0.9919803554107995
193.889592658
1.0243482917379567
204.864719692
0.9427328467213434
191.827976897
1.1257668623106396
214.259207764
0.9734020826994185
199.297217349
1.083594758672933
198.743995932
0.963777031940815
203.632720906
1.0369319084463386
210.168764564
1.01891839782148

1.4832201261995606
221.599321781
0.9816183031771408
246.77316559
0.47171262216566673
218.903449531
0.46920780697141995
223.982268915
0.42708674710997324
234.716140007
0.5367590956301576
229.720662591
0.5760485221603361
230.347322202
0.8921892576178709
250.991565554
0.9540183815912147
250.923045096
0.5849759383621094
217.596451852
0.7888476984254366
250.582068392
0.8171982939194732
246.28968382
0.4521520617221451
225.503085666
0.9535281956180257
246.57130618
0.9118707048888351
246.400824658
0.8069496186494904
243.30298553
0.5154635785870221
224.338459865
1.00080241913864
248.199971617
0.4890449332973956
227.231912413
0.4952743075570741
217.098076854
1.0384592242986537
250.633281151
0.839896239796763
249.819020149
0.40649342527692667
223.166652121
0.8997211550025366
248.695201313
0.4585497143827071
227.503846149
0.7800309072615896
242.355692933
0.5440204419185622
228.342774379
0.4855148326349955
234.615217132
0.5891579884132737
232.003868472
0.5492320745905581
219.625137564
0.40299160980

In [295]:
# Working on identifying which celestial objects land on the GFAs

from astropy.io import fits
from astropy.table import Table
import desimodel.io
import desimodel.footprint
%pylab

#sweep = Table.read('sweep-050p030-060p035.fits')
#sweep

#plot(sweep['RA'], tile['DEC'], 'b.')
tiles = desimodel.io.load_tiles()
#tiles.dtype.names
print(tiles['TILEID'])

radec = desimodel.footprint.get_tile_radec(4)
print(radec)
print(tiles['RA'])
print(tiles['DEC'])

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
[    4     6     7 ..., 46089 46090 46091]
(335.02999999999997, 19.879999999999999)
[ 335.03  333.22  332.35 ...,  249.28  251.68  254.1 ]
[ 19.88  14.84  12.32 ...,   1.43   2.9    4.34]


/anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['tile']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [296]:
def radec2xy(telra, teldec, ra, dec):
    
    """
    Returns arrays of the x, y positions of given celestial objects
    on the focal plane given an arbitrary telescope pointing in RA and Dec and
    arrays of the RA and Dec of celestial objects in the sky. Implements the Haversine formula.
    Parameters
    ----------
    telra: a scalar float signifying the telescope's RA pointing in degrees
    teldec: a scalar float signifying the telescope's Dec pointing in degrees
    ra: An array of RA values for locations in the sky
    dec: An array of declination values for locations in the sky
    """
    # Inclination is 90 degrees minus the declination in degrees
    inc = 90 - dec
    x0 = sin(math.radians(inc)) * cos(math.radians(ra))
    y0 = sin(math.radians(inc)) * sin(math.radians(ra))
    z0 = cos(math.radians(inc))
    coord = [x0, y0, z0]
    
    # Clockwise rotation around y axis by declination of the tile center
    decrotate = numpy.zeros(shape=(3,3))
    teldec_rad = math.radians(teldec)
    decrotate[0] = [cos(teldec_rad), 0, sin(teldec_rad)]
    decrotate[1] = [0, 1, 0]
    decrotate[2] = [-sin(teldec_rad), 0, cos(teldec_rad)]
    
    # Clockwise rotation around the z-axis by the right ascension of the tile center
    rarotate = numpy.zeros(shape=(3,3))
    telra_rad = math.radians(telra)
    rarotate[0] = [cos(telra_rad), sin(telra_rad), 0]
    rarotate[1] = [-sin(telra_rad), cos(telra_rad), 0]
    rarotate[2] = [0, 0, 1]
    
    #coord1 = matmul(decrotate, coord)
    #coord2 = matmul(rarotate, coord1)
    
    coord1 = matmul(rarotate, coord)
    coord2 = matmul(decrotate, coord1)
    x = coord2[0]
    y = coord2[1]
    z = coord2[2]
    print(x, y, z)
    
    theta_deg = np.sqrt(y**2 + z**2)
    theta_rad = theta_deg * 180 / np.pi
    #radius = get_radius_mm(theta_rad)
    
    
    p = np.array([8.297E5, -1750.0, 1.394E4, 0.0])
    radius = 0.0
    for i in range(4):
        radius = theta_deg*radius + p[i]


    print(radius)
    testx = radius * -y / theta_deg
    testy = radius * -z / theta_deg
    
    
    return testx, testy
    
    
"""newra, newdec = xy2radec(8.37, -10.65, -138.345, -333.179)
8.927313423598427
-9.324956250231294"""
x, y = radec2xy(8.37, -10.65, 8.927313423598427, -9.324956250231294)
print(x)
print(y)
r = get_radius_deg(x, y)
print(r)
q = cartesian_to_polar_angle(x, y)
print(q)

"""0.99968672009 0.00959826341995 0.0231156876504
360.7194471799543
-138.329551039
-333.141793961
1.4340684672282693"""



    
    

0.99968672009 0.00959826341995 0.0231156876504
360.820536724
-138.368317097
-333.235155026
1.4344442883176416
247.450443362


'0.99968672009 0.00959826341995 0.0231156876504\n360.7194471799543\n-138.329551039\n-333.141793961\n1.4340684672282693'

In [299]:
# Implements the haversine formula 
def radec2xy(telra, teldec, ra, dec):
    """
    Returns arrays of the x, y positions of given celestial objects
    on the focal plane given an arbitrary telescope pointing in RA and Dec and
    arrays of  the RA and Dec of celestial objects in the sky
    Parameters
    ----------
    telra: a scalar float signifying the telescope's RA pointing in degrees
    teldec: a scalar float signifying the telescope's Dec pointing in degrees
    ra: An array of RA values for locations in the sky
    dec: An array of declination values for locations in the sky
    """
    import numpy as np
    import math
    # Inclination is 90 degrees minus the declination in degrees
    dec = np.asarray(dec)
    inc = 90 - dec
    ra = np.asarray(ra)
    #inc = 90 - dec
    x0 = np.sin(np.radians(inc)) * np.cos(np.radians(ra))
    y0 = np.sin(np.radians(inc)) * np.sin(np.radians(ra))
    z0 = np.cos(np.radians(inc))
    coord = [x0, y0, z0]
    
    # Clockwise rotation around y axis by declination of the tile center
    decrotate = np.zeros(shape=(3,3))
    teldec_rad = np.radians(teldec)
    decrotate[0] = [np.cos(teldec_rad), 0, np.sin(teldec_rad)]
    decrotate[1] = [0, 1, 0]
    decrotate[2] = [-np.sin(teldec_rad), 0, np.cos(teldec_rad)]
    
    # Clockwise rotation around the z-axis by the right ascension of the tile center
    rarotate = np.zeros(shape=(3,3))
    telra_rad = math.radians(telra)
    rarotate[0] = [np.cos(telra_rad), np.sin(telra_rad), 0]
    rarotate[1] = [-np.sin(telra_rad), np.cos(telra_rad), 0]
    rarotate[2] = [0, 0, 1]
    
    coord1 = np.matmul(rarotate, coord)
    coord2 = np.matmul(decrotate, coord1)
    x = coord2[0]
    y = coord2[1]
    z = coord2[2]
    
    newteldec = 0
    newtelra = 0
    ra_rad = np.arctan2(y, x)
    dec_rad = (np.pi / 2) - np.arccos(z / np.sqrt((x**2) + (y**2) + (z**2)))
    radius_rad = 2 * np.arcsin(np.sqrt((np.sin((dec_rad - newteldec) / 2)**2) + ((np.cos(newteldec)) * np.cos(dec_rad) * (np.sin((ra_rad - newtelra) / 2)**2))))
    radius_deg = np.degrees(radius_rad)
    
    q_rad = np.arctan2(-z, -y) 
        
    radius_mm = get_radius_mm(radius_deg)
    x_focalplane = radius_mm * np.cos(q_rad)
    y_focalplane = radius_mm * np.sin(q_rad)
    
    return x_focalplane, y_focalplane

ra = [8.40634632111, 8.927313423598427]
dec = [-9.93649291992, -9.324956250231294]

#x, y = radec2xy(8.37, -10.65, 8.927313423598427, -9.324956250231294)
x, y = radec2xy(8.37, -10.65, ra, dec)
print(x)
print(y)

#8.40634632111 -9.93649291992

"""newra, newdec = xy2radec(8.37, -10.65, -138.345, -333.179)
8.927313423598427
-9.324956250231294"""

[  -8.77510018 -138.345     ]
[-174.88049516 -333.17899999]


'newra, newdec = xy2radec(8.37, -10.65, -138.345, -333.179)\n8.927313423598427\n-9.324956250231294'